<h1><center><font size = 6>Segmenting and Clustering Neighborhoods in Toronto</center></h1>
<center>**Yao Long**</center>  
*This notebook will be mainly used for the capstone project of [Applied Data Science Specilization](https://www.coursera.org/specializations/applied-data-science)* 

## Introduction

In this lab, I will scrape this [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) to obtain the postal and borough data of Toronto. Then, I will use the Foursquare API to explore the neighborhoods of this city. I will use the **explore** function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters based on the ***k*-means** clustering algorithm. Finally, I will use the Folium library to visualize the neighborhoods in Toronto and their emerging clusters.

## Table of Contents

<span style="line-height: 1.5;">
<div class="alert alert-block alert-info" style="margin-top: 10px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>  
2. <a href="#item2">Analyze Neighborhoods in Toronto</a>  
3. <a href="#item4">Cluster Neighborhoods</a>  

</font>
</div>

In [316]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

#!pip install html5lib
import html5lib # library for reading tables from html
import json # library to handle JSON files

#!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium
import folium # map rendering library

print ("Hello Capstone Project Course!")

Hello Capstone Project Course!


## 1. Download and Explore Dataset  

Toronto has a total of 103 unique postal codes. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 103 postal codes and the corresponding neighborhoods, as well as the the latitude and logitude coordinates associated with each postal code.

We will first scrape this [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) to obtain the postal and borough data of Toronto, then download the csv file that has the geographical coordinates of each postal code from [this link](https://cocl.us/Geospatial_data) and finally build the dataframe.

#### Website scraping

Read the table into the pandas dataframe

In [317]:
d = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = d[0]
df = df[1:] 
df.columns = ["PostalCode", "Borough", "Neighborhood"]
df.head()

,PostalCode,Borough,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Delete rows with a borough that is not assigned, set the neighborhood to be the same as the borough if the neighborhood is not assigned

In [318]:
df = df[df.Borough != 'Not assigned']
df['Neighborhood'] = df.apply(
    lambda x: x['Borough'] if x['Neighborhood'] == 'Not assigned' else x['Neighborhood'],
    axis=1
)
df.head()

,PostalCode,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Combine neighborhoods with the same postal code together

In [319]:
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Quickly examine the resulting unique postal code number

In [320]:
print ('There are ' + str(df.shape[0]) + ' rows in the table')

There are 103 rows in the table


#### Geographical coordinates downloading

Download the geographical coordinates of each postal code

In [321]:
dfcor = pd.read_csv("https://cocl.us/Geospatial_data")
dfcor = dfcor.rename(columns={"Postal Code": "PostalCode"})
dfcor.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Append the coordinate data to the pandas dataframe

In [322]:
df_toronto = pd.merge(df, dfcor, on='PostalCode')
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 2. Analyze Neighborhoods in Toronto

In this section, we will first slice the original dataframe and create the map of toronto with neighborhoods superimposed on top. Then, we will take a single neighborhood as an example to explore its venues by using Foursquare API. Afterwards, we will apply the exploration process to all the neighborhoods in Toronto. Finally, we will build a new dataframe which indicates the frequency of occurrence of each venue category for all toronoto neighborhoods.

#### Toronto neighborhoods visualizing

Create a new dataframe that only contains boroughs with keyword 'Toronto'

*For simplicity, we only focus on the neighborhoods in boroughs that contain the key word 'Toronto'. *

In [323]:
toronto_data = df_toronto #[df_toronto['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Get the latitude and longitude values of toronto by the geopy library

In [324]:
address = 'Toronto'

geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top 

In [325]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Single neighborhood venue exploring

Define Foursquare Credentials and Version

In [326]:
CLIENT_ID = 'N10MDOJYJFKSSO1ZWHS1NT32KTKDS2RW1LDXB0SOBRFSGTMA' # your Foursquare ID
CLIENT_SECRET = 'SUGX40CNBYILVT3KRMICTO4FKYBCNNXFMSWHMV1APDVRYA5X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: N10MDOJYJFKSSO1ZWHS1NT32KTKDS2RW1LDXB0SOBRFSGTMA
CLIENT_SECRET:SUGX40CNBYILVT3KRMICTO4FKYBCNNXFMSWHMV1APDVRYA5X


Explore the first neighborhood in ourdataframe

In [327]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.806686299999996, -79.19435340000001.


Get the top 100 venues that are in The Beaches within a radius of 500 meters.

In [328]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

Define function that gets the category of the venue

In [329]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a *pandas* dataframe

In [330]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056


In [331]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


#### Complete Toronto neighborhoods exploring

Create a function to repeat the same process to all the neighborhoods in Toronto

In [332]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called *Toronto_venues*.

In [333]:
toronto_venues = getNearbyVenues(names = toronto_data['Neighborhood'],
                                   latitudes = toronto_data['Latitude'],
                                   longitudes = toronto_data['Longitude']
                                  )
print(toronto_venues.shape)
toronto_venues.head()

(2252, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


Check how many venues were returned for each neighborhood

In [334]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",3,3,3,3,3,3
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",11,11,11,11,11,11
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",21,21,21,21,21,21
Berczy Park,56,56,56,56,56,56


Find out how many unique categories can be curated from all the returned venues

In [335]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


#### Venue category analyzing

Count the times of occurrence of each venue category for all neighborhoods

In [336]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
idn = toronto_onehot.columns.get_loc('Neighborhood')

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[idn]] + list(toronto_onehot.columns[:idn-1]) + list(toronto_onehot.columns[idn+1:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [337]:
toronto_onehot.shape

(2252, 273)

Group rows by neighborhood

In [338]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.0,0.0,0.01,0.0,0.0,0.0,0.0,0.01,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0


In [339]:
toronto_grouped.shape

(99, 273)

Print each neighborhood along with the top 5 most common venues

In [340]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.05
2   Steakhouse  0.04
3          Bar  0.04
4          Gym  0.03


----Agincourt----
                             venue  freq
0                   Sandwich Place  0.25
1                     Skating Rink  0.25
2                   Breakfast Spot  0.25
3                           Lounge  0.25
4  Molecular Gastronomy Restaurant  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                             venue  freq
0                       Playground  0.33
1                           Bakery  0.33
2                             Park  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0        Grocery Store  0.18
1          Pizza Place  0.09
2           Beer Store  0.09
3  Fried Chicken Joint  0.

#### New dataframe building

Create a function to sort the venues in descending order

In [341]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood

In [342]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Gym,American Restaurant,Restaurant,Burger Joint,Asian Restaurant,Thai Restaurant
1,Agincourt,Lounge,Sandwich Place,Breakfast Spot,Skating Rink,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Bakery,Park,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Fried Chicken Joint,Coffee Shop,Sandwich Place,Discount Store,Fast Food Restaurant,Beer Store,Japanese Restaurant,Pharmacy
4,"Alderwood, Long Branch",Pizza Place,Skating Rink,Pharmacy,Coffee Shop,Pub,Sandwich Place,Gym,Airport Service,Falafel Restaurant,Ethiopian Restaurant


## 3. Cluster Neighborhoods

In this section, we will first cluster the neighborhoods into 5 clusters based on *k*-means clustering. Then, we will visualize the resulting clusters. Finally, we will run an examination of the obtained clusters.

#### *k*-means clustering

Run *k*-means to cluster the neighborhood into 5 clusters.

In [343]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
np.int_(kmeans.labels_)

array([0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 2, 0, 1, 0, 2, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1])

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [345]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', np.int_(kmeans.labels_))

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,2.0,Fast Food Restaurant,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,0.0,Construction & Landscaping,Bar,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,0.0,Pizza Place,Mexican Restaurant,Electronics Store,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Diner,Discount Store,Dive Bar
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Diner,Dive Bar,Dog Run


In [354]:
toronto_merged = toronto_merged.dropna(axis = 0, how ='any') 
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(np.int64)
toronto_merged.shape

(99, 16)

#### Clusters visualizing

In [355]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Clusters examing

##### Cluster 1

In [356]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,Construction & Landscaping,Bar,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,Scarborough,0,Pizza Place,Mexican Restaurant,Electronics Store,Breakfast Spot,Intersection,Medical Center,Rental Car Location,Diner,Discount Store,Dive Bar
3,Scarborough,0,Coffee Shop,Korean Restaurant,Dumpling Restaurant,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
4,Scarborough,0,Hakka Restaurant,Bakery,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Fried Chicken Joint,Diner,Dive Bar,Dog Run
6,Scarborough,0,Discount Store,Department Store,Bus Station,Hobby Shop,Coffee Shop,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
7,Scarborough,0,Bakery,Bus Line,Soccer Field,Metro Station,Fast Food Restaurant,Intersection,Bus Station,Dog Run,Doner Restaurant,Donut Shop
8,Scarborough,0,Motel,American Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dessert Shop
9,Scarborough,0,College Stadium,General Entertainment,Skating Rink,Café,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
10,Scarborough,0,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,Comic Shop,Dim Sum Restaurant,Ethiopian Restaurant,Empanada Restaurant,Colombian Restaurant
11,Scarborough,0,Breakfast Spot,Bakery,Smoke Shop,Middle Eastern Restaurant,Women's Store,Donut Shop,Dive Bar,Dog Run,Doner Restaurant,Dumpling Restaurant


##### Cluster 2

In [357]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,North York,1,Park,Bank,Convenience Store,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
30,North York,1,Airport,Park,Construction & Landscaping,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
31,North York,1,Grocery Store,Bank,Park,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
40,East York,1,Park,Pizza Place,Coffee Shop,Rental Car Location,Convenience Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dive Bar
44,Central Toronto,1,Park,Bus Line,Swim School,Women's Store,Donut Shop,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
48,Central Toronto,1,Summer Camp,Park,Playground,Tennis Court,Women's Store,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant
50,Downtown Toronto,1,Park,Building,Playground,Trail,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run
72,North York,1,Park,Asian Restaurant,Japanese Restaurant,Pub,Women's Store,Doner Restaurant,Diner,Discount Store,Dive Bar,Dog Run
74,York,1,Park,Women's Store,Fast Food Restaurant,Market,Pharmacy,Gourmet Shop,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
79,North York,1,Bakery,Park,Construction & Landscaping,Basketball Court,Women's Store,Drugstore,Dive Bar,Dog Run,Doner Restaurant,Donut Shop


##### Cluster 3

In [358]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,2,Fast Food Restaurant,Drugstore,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Wings Joint
25,North York,2,Food & Drink Shop,Fast Food Restaurant,Park,Women's Store,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop


##### Cluster 4

In [359]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,3,Pool,Baseball Field,Women's Store,Donut Shop,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore
97,North York,3,Construction & Landscaping,Baseball Field,Women's Store,Dumpling Restaurant,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


##### Cluster 5

In [360]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,4,Playground,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Drugstore,College Rec Center
14,Scarborough,4,Playground,Bakery,Park,Women's Store,Drugstore,Discount Store,Dive Bar,Dog Run,Doner Restaurant,Donut Shop
